In [2]:
#Importing all these tools.  I usually need one or the other.
#For now lets just grab them all.  Some might not be used.
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import explore
import warnings
warnings.filterwarnings("ignore")

### Given the following confusion matrix, evaluate (by hand) the model's performance.

In [3]:
data = {'actual_dog':['46','7'],'actual_cat':['13','34']}
df = pd.DataFrame(data, columns = ['actual_dog','actual_cat'], index=['pred_dog','pred_cat'])
df

,actual_dog,actual_cat
pred_dog,46,13
pred_cat,7,34


#### In the context of this problem, what is a false positive?
    False positive would be Not a Dog but predicted a Dog
#### In the context of this problem, what is a false negative?
    False negative is that IS a Dog but predicted a Cat
#### How would you describe this model?
    Crosstab of predicting dogs and cats

### You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

Use the predictions dataset and pandas to help answer the following questions:

### An internal team wants to investigate the cause of the manufacturing defects. 

In [4]:
df = pd.read_csv("c3.csv")

In [5]:
df.head(3)

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect


In [6]:
df.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

In [7]:
pd.crosstab(df.model1, df.actual)

actual,Defect,No Defect
model1,,
Defect,8,2
No Defect,8,182


Postive = Defects<br>
Negative = No Defects

TP = Actual has Defects and Model has Defects<br>
FP = Actual has no Defects but Model has Defects<br>
FN = Actual has Defects but Model has no Defects<br>
TN = Actual has no Defects and Model has no Defects

### They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [8]:
df['baseline_prediction'] = 'No Defect' #Most frequent label in actual
df.head(3)

,actual,model1,model2,model3,baseline_prediction
0,No Defect,No Defect,Defect,No Defect,No Defect
1,No Defect,No Defect,Defect,Defect,No Defect
2,No Defect,No Defect,Defect,No Defect,No Defect


In [9]:
# Recall - calculate on subset of data
# only look at subset of actual positives (i.e actual == coffee)
subset = df[df.actual == 'Defect'] #postive 
subset.head(3)

,actual,model1,model2,model3,baseline_prediction
13,Defect,No Defect,Defect,Defect,No Defect
30,Defect,Defect,No Defect,Defect,No Defect
65,Defect,Defect,Defect,Defect,No Defect


In [10]:
model_recall = (subset.actual == subset.model1).mean()
print(f'   model1 recall: {model_recall:.2%}')

model_recall2 = (subset.actual == subset.model2).mean()
print(f'   model2 recall: {model_recall2:.2%}')

model_recall3 = (subset.actual == subset.model3).mean()
print(f'   model3 recall: {model_recall3:.2%}')

   model1 recall: 50.00%
   model2 recall: 56.25%
   model3 recall: 81.25%


### Which evaluation metric would be appropriate here? 
    I would use recall for the evaluation metric.

### Which model would be the best fit for this use case?
    I would pick the model with the most defects.  In our duck case, I would pick Model 3 @ 81% positive occurances

### Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect.
    My first thought is how do I get my hands on one of these defective ducks

In [16]:
#TP/(TP + FP)
#Actual has no Defects but Model has Defects
subset = df[df.model1 == 'Defect']
subset.head(3)

,actual,model1,model2,model3,baseline_prediction
3,No Defect,Defect,Defect,Defect,No Defect
30,Defect,Defect,No Defect,Defect,No Defect
62,No Defect,Defect,No Defect,No Defect,No Defect


In [17]:
model_precision = (subset.actual == subset.model1).mean()
print(f'model1 precision: {model_precision:.2%}')

model1 precision: 80.00%


In [18]:
subset = df[df.model2 == 'Defect']
model_precision = (subset.actual == subset.model2).mean()
print(f'model2 precision: {model_precision:.2%}')

model2 precision: 10.00%


In [19]:
subset = df[df.model3 == 'Defect']
model_precision = (subset.actual == subset.model3).mean()
print(f'model3 precision: {model_precision:.2%}')

model3 precision: 13.13%


### Which evaluation metric would be appropriate here?
    I would use precision to get only the ones that are defects and not any of the ones that were false
### Which model would be the best fit for this use case?
    I would use Model 1. It had the highest precision

You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

Several models have already been developed with the data, and you can find their results here.

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [20]:
df = pd.read_csv("gives_you_paws.csv")

In [21]:
df.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [24]:
df.actual.value_counts()

dog    3254
cat    1746
Name: actual, dtype: int64

In [26]:
pd.crosstab(df.model1, df.actual)

actual,cat,dog
model1,,
cat,1423,640
dog,323,2614


Positive is a cat<br>
Negative is Not a cat

TP =is a Cat, Predicted Cat<br>
FN =is a Cat, Predicted not a cat<br>
FP =is not a Cat. Predicted Cat<br>
TN =is not a Cat. Predicted not a Cat<br>

### In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [34]:
df['baseline_prediction'] = 'dog' #Most frequent label in actual
df.head(3)

,actual,model1,model2,model3,model4,baseline_prediction
0,cat,cat,dog,cat,dog,dog
1,dog,dog,cat,cat,dog,dog
2,dog,cat,cat,cat,dog,dog


In [35]:
accuracy_subset = df[df.actual == 'cat'] #postive 
accuracy_subset.head(3)

,actual,model1,model2,model3,model4,baseline_prediction
0,cat,cat,dog,cat,dog,dog
4,cat,cat,cat,dog,dog,dog
6,cat,cat,cat,cat,dog,dog


In [37]:
model_accuracy = (df.actual == df.model1).mean()
baseline_accuracy = (df.actual == df.baseline_prediction).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}\n')

model_accuracy = (df.actual == df.model2).mean()
baseline_accuracy = (df.actual == df.baseline_prediction).mean()

print(f'   mode2 accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}\n')

model_accuracy = (df.actual == df.model3).mean()
baseline_accuracy = (df.actual == df.baseline_prediction).mean()

print(f'   mode3 accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}\n')

model_accuracy = (df.actual == df.model4).mean()
baseline_accuracy = (df.actual == df.baseline_prediction).mean()

print(f'   model4 accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model accuracy: 80.74%
baseline accuracy: 65.08%

   mode2 accuracy: 63.04%
baseline accuracy: 65.08%

   mode3 accuracy: 50.96%
baseline accuracy: 65.08%

   model4 accuracy: 74.26%
baseline accuracy: 65.08%


### In terms of accuracy, how do the various models compare to the baseline model?
    Half of the model Accuracy's are higher than baseline
### Are any of the models better than the baseline?
    Yes half of them

### Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? For Phase II?

### Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I? For Phase II?